# [캡스톤-04] 선형회귀ㅣ💎 다이아몬드 가격 결정 구조 분석

> 아이티윌 데이터분석 부트캠프 52기<br/>
> 이광호 강사 (leekh4232@gmail.com)

본 과제는 Kaggle의 Diamonds 데이터셋을 활용하여
데이터 전처리부터 탐색적 데이터 분석, 통계적 추론, 회귀분석까지
데이터 분석의 전체 흐름을 경험하는 것을 목표로 하는 평가 과제 입니다.

아래 미션들은 “정답을 맞히는 과제”가 아니라,
**데이터를 통해 하나의 이야기를 만들어 가는 탐구 과제**입니다.

각 미션은 서로 독립적이지만, 모두 합치면 하나의 질문으로 수렴합니다.

> **“다이아몬드 가격은 왜 이렇게 결정되는 걸까?”**

이 과제의 목적은 “회귀식 = 숫자”가 아니라 **“회귀식 = 설명 가능한 세계관”** 이라는 인식을 심어주는 데 목적이 있습니다.

※ 본 과제는 팀/개인 단위 모두 수행 가능합니다.

## 데이터 불러오기

```python
load_data("diamonds")
```

## 데이터 설명

| field   | description                                                    |
|---------|----------------------------------------------------------------|
| price   | 다이아몬드 가격 (USD, $326 ~ $18,823)                                 |
| carat   | 중량 (0.2~5.01)                                                  |
| cut     | 컷 품질 (Fair, Good, Very Good, Premium, Ideal)                   |
| color   | 색상 등급 - J (worst) to D (best)                                  |
| clarity | 투명도 등급 (I1 (worst), SI2, SI1, VS2, VS1, VVS2, VVS1, IF (best)) |
| x       | 길이 mm (0~10.74)                                                |
| y       | 너비 mm (0--58.9)                                                |
| z       | 두께 mm (0--31.8)                                                |
| depth   | 비율 정보 = z / mean(x, y) = 2 * z / (x + y) (43--79)              |
| table   | 다이아몬드의 가장 넓은 지점에 비해 상단(테이블)의 너비 (43--95)                       |

## 준비작업

### 패키지 참조

In [1]:
from hossam import *
from pandas import DataFrame, concat

### 데이터 불러오기

In [2]:
origin = hs_util.load_data("diamonds", categories=["cut", "color", "clarity"])

[data] https://data.hossam.kr/data/kaggle/diamonds.xlsx
[desc] 다이아몬드를 커팅, 색상, 투명도, 가격 및 기타 속성 조사한 데이터 셋 (출처: https://www.kaggle.com/datasets/shivam2503/diamonds)

[!] Cannot read metadata

✅ 테이블 정보
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53940 entries, 0 to 53939
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype   
---  ------   --------------  -----   
 0   price    53940 non-null  int64   
 1   carat    53940 non-null  float64 
 2   cut      53940 non-null  category
 3   color    53940 non-null  category
 4   clarity  53940 non-null  category
 5   x        53940 non-null  float64 
 6   y        53940 non-null  float64 
 7   z        53940 non-null  float64 
 8   depth    53940 non-null  float64 
 9   table    53940 non-null  float64 
dtypes: category(3), float64(6), int64(1)
memory usage: 3.0 MB


✅ 상위 5개 행
+---+-------+-------+---------+-------+---------+------+------+------+-------+-------+
|   | price | carat |   cut   | color | clarity |  x   |  y   |

## 미션 1. **“이 데이터는 얼마나 믿을 수 있을까?”**

본격적인 분석에 들어가기 전에,
우리가 사용할 데이터가 **얼마나 신뢰할 수 있는지**부터 점검해 보자.

데이터 안에는
- 비어 있는 값은 없는지,
- 유난히 튀는 값은 없는지,
- 현실적으로 말이 되지 않는 값은 없는지
차근차근 살펴볼 필요가 있다.

예를 들어, 다이아몬드의 크기를 나타내는 `x`, `y`, `z` 값이
0으로 기록된 경우가 있는지 직접 확인해 보자.
만약 있다면, 이런 값들을 그대로 분석에 사용해도 괜찮을지 팀에서 논의해 보자.

또한 `cut`, `color`, `clarity`와 같은 품질 변수들이
몇 개의 범주로 구성되어 있고,
특정 범주에 데이터가 지나치게 몰려 있지는 않은지도 함께 확인해 보자.

마지막으로, 전처리 전과 후를 비교하며
**어떤 기준으로 데이터를 제거하거나 수정했는지**를 정리해 보자.

> 📌 **출제 의도**
> 분석 전에 “이 값이 말이 되나?”를 먼저 묻는 습관을 들입니다. 결측·이상치 처리 기준을 말로 남겨야 이후 회귀 해석이 설득력을 가집니다.

### 1-1. 결측값 검증

In [3]:
hs_stats.missing_values(origin)

,missing_count,missing_rate
field,,
price,0,0.0
carat,0,0.0
cut,0,0.0
color,0,0.0
clarity,0,0.0
x,0,0.0
y,0,0.0
z,0,0.0
depth,0,0.0


#### 인사이트
- 출력 결과 모든 컬럼의 결측값 개수와 비율이 0으로 확인되었다.
- 따라서 결측값 처리로 인한 표본 손실 없이 다음 단계(현실성/이상치 검증)로 진행할 수 있다.


### 1-2. 현실성 검증 (물리적 불가능한 값)

In [4]:
# x, y, z의 0 또는 음수 값 확인 (물리적으로 불가능)
size_cols = ["x", "y", "z"]
zero_counts = (origin[size_cols] == 0).sum()
zero_any = (origin[size_cols] == 0).any(axis=1).sum()
neg_any = (origin[size_cols] < 0).any(axis=1).sum()

print("\n===== 크기 변수 0 값 개수 =====")
print(zero_counts)
print(f"\n0 값이 하나라도 있는 행 수: {int(zero_any)}")
print(f"음수 값이 있는 행 수: {int(neg_any)}")

# 0 값이 있는 샘플 확인
origin.loc[(origin[size_cols] == 0).any(axis=1), size_cols].head()


===== 크기 변수 0 값 개수 =====
x     8
y     7
z    20
dtype: int64

0 값이 하나라도 있는 행 수: 20
음수 값이 있는 행 수: 0


,x,y,z
2207,6.55,6.48,0.0
2314,6.66,6.60,0.0
4791,6.50,6.47,0.0
5471,6.50,6.47,0.0
10167,7.15,7.04,0.0


#### 인사이트
- x/y/z 중 하나라도 0인 행이 20개 존재했고, 음수 값은 없었다.
- x, y, z는 물리적 크기이므로 0은 비정상값으로 간주되며, 이후 전처리 단계에서 제거 또는 수정 기준을 명시해야 한다.


### 1-3. 이상치 감지

In [5]:
outlier_table = hs_stats.outlier_table(origin)
outlier_table.sort_values("outlier_rate", ascending=False)

,q1,q2,q3,iqr,up,down,min,max,skew,outlier_count,outlier_rate
field,,,,,,,,,,,
price,950.00,2401.00,5324.25,4374.25,11885.625,-5611.375,326.0,18823.00,1.618395,3540,6.562848
depth,61.00,61.80,62.50,1.50,64.750,58.750,43.0,79.00,-0.082294,2545,4.718205
carat,0.40,0.70,1.04,0.64,2.000,-0.560,0.2,5.01,1.116646,1889,3.502039
table,56.00,57.00,59.00,3.00,63.500,51.500,43.0,95.00,0.796896,605,1.121617
z,2.91,3.53,4.04,1.13,5.735,1.215,0.0,31.80,1.522423,49,0.090842
x,4.71,5.70,6.54,1.83,9.285,1.965,0.0,10.74,0.378676,32,0.059325
y,4.72,5.71,6.54,1.82,9.270,1.990,0.0,58.90,2.434167,29,0.053763


#### 인사이트

- 가격(price)과 깊이(depth), 중량(carat)에서 이상치가 특히 많이 나타나므로, 제거/변환 기준을 이 변수들 중심으로 설정하는 것이 합리적이다.


### 1-4. 범주형 변수 구성

In [6]:
hs_stats.category_table(origin)

count       rate
field   category                   
cut     Ideal      21551  39.953652
        Premium    13791  25.567297
        Very Good  12082  22.398962
        Good        4906   9.095291
        Fair        1610   2.984798
color   G          11292  20.934372
        E           9797  18.162773
        F           9542  17.690026
        H           8304  15.394883
        D           6775  12.560252
        I           5422  10.051910
        J           2808   5.205784
clarity SI1        13065  24.221357
        VS2        12258  22.725250
        SI2         9194  17.044865
        VS1         8171  15.148313
        VVS2        5066   9.391917
        VVS1        3655   6.776047
        IF          1790   3.318502
        I1           741   1.373749

#### 인사이트
- cut은 5개, color는 7개, clarity는 8개 범주로 구성되어 있다.
- cut은 Ideal이 가장 많고 Fair가 가장 적으며, clarity는 SI1이 가장 많고 I1이 가장 적어 범주 간 분포 차이가 크다.


### 1-5. 범주별 분포 편향

In [ ]:
hs_stats.category_summary(origin)

,변수,최다_범주,최다_비율(%),최소_범주,최소_비율(%)
0,cut,Ideal,39.95,Fair,2.98
1,color,G,20.93,J,5.21
2,clarity,SI1,24.22,I1,1.37


#### 인사이트
- cut은 Ideal(39.95%)에 집중되고 Fair(2.98%)가 매우 적어 편향이 뚜렷하다.
- color는 G(20.93%)가 가장 많고 J(5.21%)가 가장 적으며, clarity는 SI1(24.22%)가 가장 많고 I1(1.37%)가 가장 적다.


### 1-6. 전처리 기준 및 영향 정리

In [ ]:
# 전처리 기준 적용(예: x/y/z=0 제거) 및 표본 영향 확인
size_cols = ["x", "y", "z"]
invalid_mask = (origin[size_cols] == 0).any(axis=1)
before_n = len(origin)
after_n = before_n - invalid_mask.sum()
removed_n = int(invalid_mask.sum())
removed_ratio = removed_n / before_n * 100

print("\n===== 전처리 기준 적용 요약 =====")
print("제거 기준: x/y/z 중 하나라도 0")
print(f"제거 행 수: {removed_n}")
print(f"전체 대비 비율: {removed_ratio:.2f}%")
print(f"전처리 전/후 데이터 크기: {before_n} -> {after_n}")

#### 인사이트
- 물리적으로 불가능한 크기값(x/y/z=0) 20건을 제거하면 표본의 약 0.04%만 감소한다.
- IQR 이상치는 전체의 약 11.89%로 크기 때문에, 즉시 제거하기보다 로그 변환/로버스트 기법 등 대안과 함께 판단하는 것이 합리적이다.


## 미션 2. **“가격 데이터는 어떤 모습일까?”**

`price` 변수를 처음 마주했을 때의 분포를 직접 그려보자.
가격이 고르게 분포되어 있는지,
아니면 특정 구간에 몰려 있는지 관찰해 보자.

이어서 `carat` 변수의 분포도 함께 살펴보고,
두 변수의 공통점과 차이점을 말로 설명해 보자.

분포를 살펴본 뒤에는
“이 상태로 회귀분석을 해도 괜찮을까?”라는 질문을 던져보자.

> 📌 **출제 의도**
> 목표 변수와 핵심 설명 변수의 생김새를 먼저 확인해, 지표 선택과 변환(로그 등) 필요성을 스스로 느끼게 하기 위함입니다.

### 2-1. price 분포 분석

In [ ]:
hs_plot.kde_confidence_interval(origin, xnames="price")

#### 인사이트

### 📊 다이아몬드 가격 분포 그래프에서 확인되는 객관적 사실

- 가격 분포는 **단봉형(unimodal)** 구조를 가진다.
- 분포의 **최대 밀도(피크)**는 저가 구간(대략 수천 달러 이하)에 위치한다.
- 가격 분포는 **오른쪽 꼬리가 긴(right-skewed)** 형태를 보인다.
- 고가로 갈수록 밀도는 **점진적으로 감소**하며, 급격한 절벽 형태는 나타나지 않는다.
- **$15,000 이상 구간에서도 밀도가 0은 아니며**, 매우 소수의 고가 관측치가 존재한다.
- 분포의 중심부 대비 고가 구간의 **밀도 값은 현저히 낮다**.
- 가격이 증가함에 따라 **밀도 감소가 단조적으로 진행**되는 구간이 길게 이어진다.
- 표시된 세로 점선 구간 **[price] 3899.1 ~ 3966.5**는  
  - 분포의 **최대 피크보다 오른쪽에 위치**한다.  
  - 가장 많은 관측치가 몰린 구간과는 **일치하지 않는다**.
- 평균(또는 중심값으로 표시된 구간)은 **분포의 비대칭성으로 인해 피크보다 오른쪽에 위치**한다.
- 분포의 좌측(저가 방향)은 비교적 **짧고 급격히 상승**하는 형태를 보인다.
- 분포의 우측(고가 방향)은 **완만하게 감소하는 긴 꼬리** 형태를 가진다.
- 동일한 밀도 높이를 기준으로 볼 때,  
  - 저가 구간은 **짧은 가격 범위에 밀집**되어 있고  
  - 고가 구간은 **넓은 가격 범위에 희박하게 분산**되어 있다.
- 가격 값의 전체 범위는 **수백 달러대부터 약 2만 달러 수준까지**로 매우 넓다.
- 그래프 상에서 **명확한 이중 피크(bimodal) 구조는 관측되지 않는다**.
- 분포의 대부분의 면적은 **저가~중가 구간에 집중**되어 있다.
- 평균 구간이 위치한 가격대는 **분포의 중앙이 아니라 상위 쪽에 해당**한다.

**그러므로 로그 변환을 고려해야 한다**

### 2-2. carat 분포 분석

In [ ]:
hs_plot.kde_confidence_interval(origin, xnames="carat")

#### 인사이트

- 가격 분포는 **단봉형(unimodal)** 구조를 가진다.
- 분포의 **최대 밀도(피크)**는 저가 구간(대략 수천 달러 이하)에 위치한다.
- 가격 분포는 **오른쪽 꼬리가 긴(right-skewed)** 형태를 보인다.
- 고가로 갈수록 밀도는 **점진적으로 감소**하며, 급격한 절벽 형태는 나타나지 않는다.
- **$15,000 이상 구간에서도 밀도가 0은 아니며**, 매우 소수의 고가 관측치가 존재한다.
- 분포의 중심부 대비 고가 구간의 **밀도 값은 현저히 낮다**.
- 가격이 증가함에 따라 **밀도 감소가 단조적으로 진행**되는 구간이 길게 이어진다.
- 표시된 세로 점선 구간 **[price] 3899.1 ~ 3966.5**는  
  - 분포의 **최대 피크보다 오른쪽에 위치**한다.  
  - 가장 많은 관측치가 몰린 구간과는 **일치하지 않는다**.
- 평균(또는 중심값으로 표시된 구간)은 **분포의 비대칭성으로 인해 피크보다 오른쪽에 위치**한다.
- 분포의 좌측(저가 방향)은 비교적 **짧고 급격히 상승**하는 형태를 보인다.
- 분포의 우측(고가 방향)은 **완만하게 감소하는 긴 꼬리** 형태를 가진다.
- 동일한 밀도 높이를 기준으로 볼 때,  
  - 저가 구간은 **짧은 가격 범위에 밀집**되어 있고  
  - 고가 구간은 **넓은 가격 범위에 희박하게 분산**되어 있다.
- 가격 값의 전체 범위는 **수백 달러대부터 약 2만 달러 수준까지**로 매우 넓다.
- 그래프 상에서 **명확한 이중 피크(bimodal) 구조는 관측되지 않는다**.
- 분포의 대부분의 면적은 **저가~중가 구간에 집중**되어 있다.
- 평균 구간이 위치한 가격대는 **분포의 중앙이 아니라 상위 쪽에 해당**한다.

**그러므로 로그 변환을 고려해야 한다**

### 2-3. 분포 비교 분석 (코드 없음. 서술형)

| 구분             | price (가격)                  | carat (캐럿)                    |
| -------------- | --------------------------- | ----------------------------- |
| 변수 유형          | 연속형 수치 변수                   | 연속형이지만 **이산적 특성**을 함께 가짐      |
| 전체 분포 형태       | 단봉형(unimodal)에 가까움          | 다봉형에 가까움 (여러 국소적 피크 존재)       |
| 분포 비대칭성        | 강한 **우측 비대칭(right-skewed)** | 강한 **우측 비대칭(right-skewed)**   |
| 밀도 집중 구간       | 저가 구간에 밀도 집중                | 1캐럿 이하 구간에 밀도 집중              |
| 평균(기준선) 위치     | 최빈 구간보다 **오른쪽**에 위치         | 최대 피크보다 **오른쪽**에 위치           |
| 평균의 대표성        | 소수의 고가 값에 의해 평균이 크게 이동      | 특정 규격 캐럿 값의 영향으로 평균 왜곡        |
| 꼬리(tail) 특성    | 고가 방향으로 **매우 긴 꼬리** 유지      | 2캐럿 이후 급격히 희박해짐               |
| 극단값의 영향        | 고가 관측치가 분포 전체에 지속적 영향       | 극단값은 존재하나 매우 희소               |
| 분포의 매끄러움       | 비교적 **매끄러운 연속 분포**          | **톱니형·계단형 분포**                |
| 국소적 피크         | 뚜렷하지 않음                     | 0.3, 0.5, 1.0, 2.0 캐럿 부근에서 관측 |
| 값 증가에 따른 밀도 변화 | 값 증가에 따라 점진적 감소             | 값 증가에 따라 불규칙적 감소              |
| 절대 변화의 해석      | 동일 금액 증가의 의미가 구간별로 다름       | 동일 캐럿 증가가 비선형적 의미를 가짐         |
| 로그 변환 필요성 근거   | 긴 꼬리·평균 왜곡·극단값 영향           | 비대칭성·스케일 불균형·피크 압축 필요         |


### 2-4. 변환 필요성 논의 (코드 없음. 서술형)

- price와 carat 모두 오른쪽 꼬리가 길고 평균이 중앙값보다 큰 비대칭 분포다.
- 이런 분포는 회귀에서 잔차 정규성/등분산성 가정을 흔들 수 있어 로그 변환을 고려하는 것이 타당하다.


## 미션 3. **“로그 변환은 왜 등장할까?”**

`price`와 `carat`에 로그 변환을 적용해 보고,
변환 전과 후의 분포를 나란히 비교해 보자.

로그 변환을 하면 무엇이 달라졌는지,
그리고 왜 많은 분석에서 로그 변환을 사용하는지
이번 데이터셋을 기준으로 설명해 보자.

이때 중요한 것은
“로그를 썼다”가 아니라
**“왜 써야 했는가”**이다.

> 📌 **출제 의도**
> 단순한 기법 나열이 아니라 “왜 이 변환이 필요한가”를 설명하는 연습입니다. 분포와 해석이 어떻게 달라지는지 체감해 보세요.

### 3-1. price의 로그 변환

In [ ]:
df1 = hs_prep.log_transform(origin, "price")

hs_plot.kde_confidence_interval(df1, xnames="price")

#### 인사이트

- 로그 변환 후 분포는 **원래의 강한 우측 비대칭(right-skewed)** 구조가 **상당히 완화**되었다.
- 변환 전 price 분포에서 관측되던 **긴 오른쪽 꼬리**가 로그 변환 후에는 **짧아지고 압축된 형태**로 나타난다.
- 로그 변환 후 분포는 **좌·우 밀도의 균형이 개선**되어, 시각적으로 더 **대칭적인 구조**에 가까워졌다.
- 변환 전에는 고가 구간의 소수 관측치가 분포의 형태를 지배했으나, 로그 변환 후에는 **고가 관측치의 영향력이 상대적으로 감소**하였다.
- 로그 변환 후에도 분포는 **완전히 단봉형은 아니며**, **여러 개의 국소적 피크(local peaks)**가 관측된다.
- 이는 가격이 **연속적인 하나의 집단이 아니라**, **여러 가격대 집단이 혼합된 구조**임을 시사한다.
- 평균(또는 기준선으로 표시된 약 7.8)은 변환 전보다 **분포의 중심부에 더 가까운 위치**로 이동하였다.
- 변환 전 price 분포에서는 평균이 최빈 구간과 크게 어긋나 있었으나, 로그 변환 후에는 **평균과 고밀도 구간 간 괴리가 감소**하였다.
- 로그 변환 후 분포에서는 **저가 구간의 과도한 밀집 현상이 완화**되고, 가격대별 밀도 차이가 보다 **완만하게 표현**된다.
- 분포의 전체 범위는 로그 스케일 상에서 **압축**되어, 극단적인 가격 차이가 **상대적 차이(비율)**로 재표현된다.
- 로그 변환 결과,**분포의 해상도가 저가·고가 구간 간 보다 균등하게 배분**된다.
- 이는 회귀 분석 시 특정 가격 구간에 잔차가 과도하게 집중될 가능성을 **완화할 수 있는 형태**이다.
- 변환 전 분포에서는 “절대 가격 차이”가 강조되었으나, 로그 변환 후 분포에서는 **가격의 상대적 차이 구조**가 더 명확하게 드러난다.
- 결과적으로, 로그 변환은 price 변수의 분포를 **회귀 분석에 더 적합한 형태로 정규화 방향으로 이동**시킨다.

### 3-2. carat의 로그 변환

In [ ]:
df2 = hs_prep.log_transform(df1, "carat")
hs_plot.kde_confidence_interval(df2, xnames="carat")

#### 인사이트

- 로그 변환 전 carat 분포에서 관측되던 **강한 우측 비대칭(right-skewed)** 구조가 로그 변환 후에는 **상당 부분 완화**되었다.
- 변환 전에는 1캐럿 이하 구간에 밀도가 과도하게 집중되어 있었으나, 로그 변환 후에는 **저·중·고 캐럿 구간 간 밀도 분포가 보다 균등하게 분산**되었다.
- 변환 전 분포에서 길게 늘어져 있던 **고캐럿 방향 꼬리**는 로그 변환 후 **짧고 압축된 형태**로 나타난다.
- 로그 변환 후에도 분포는 **완전히 단봉형으로 수렴하지 않으며**, **여러 개의 국소적 피크(local peaks)**가 여전히 관측된다.
- 이는 carat 값이 본질적으로 **연속적이면서도 특정 규격값에 밀집되는 구조**를 가지고 있음을 보여준다.
- 로그 변환 전 분포에서는 동일한 0.1캐럿 증가의 의미가 구간마다 달랐으나, 로그 변환 후에는 **상대적 변화(비율 변화)**로 표현되어 해석 단위가 일관된다.
- 표시된 기준선(약 −0.4)은 로그 변환 전 기준값(약 0.8캐럿)에 비해 **분포의 중심부에 더 가까운 위치**에 놓인다.
- 로그 변환 후 분포에서는 극단적으로 큰 캐럿 값의 영향력이 **시각적으로 완화**되었다.
- 다만, 로그 변환으로 인해 **소형 캐럿 구간의 미세한 차이는 상대적으로 확대되어 표현**된다.
- 변환 전 분포에서는 분포의 대부분이 한쪽에 몰려 있었으나, 로그 변환 후에는 **분포 전체 구간의 해상도가 개선**되었다.
- 결과적으로 로그 변환은 carat 변수의 스케일 불균형을 완화하였으나, **이산적 규격값으로 인한 다봉 구조 자체를 제거하지는 않는다.**
- 이는 carat 변수가 회귀 분석에서 **로그/비로그 중 선택의 여지가 있는 변수**임을 시각적으로 뒷받침한다.

### 3-3. 로그 변환의 필요성 및 해석 변화 (코드 없음. 서술형)

- price와 carat 모두 우측 비대칭 분포로 극단값의 영향이 큼
- 원자료에서는 평균이 분포의 중심을 잘 대표하지 못함
- 로그 변환은 긴 꼬리를 압축하여 분포의 안정성을 높임
- 극단적으로 큰 값이 회귀 결과에 미치는 영향이 완화됨
- 값의 크기에 따른 분산 차이가 줄어듦
- 해석 기준이 절대 변화 → 상대(비율) 변화로 전환됨
- 회귀계수는 “단위 증가 효과”가 아니라 % 변화 효과로 해석됨
- 평균 기반 회귀 해석이 보다 일반적인 관측치에 가까워짐
- 다만 로그 변환이 모든 문제를 해결하는 것은 아니며, 분포 개선과 해석 전환을 위한 도구로 이해해야 함

## 미션 4. **“품질 등급은 가격을 얼마나 설명해 줄까?”**

`cut`, `color`, `clarity`에 따라
다이아몬드 가격 분포가 어떻게 달라지는지 시각화해 보자.

중앙값, 분산, 분포의 겹침 정도를 관찰하며 다음 질문에 답해 보자.

- 어떤 품질 변수는 가격 차이가 뚜렷한가?
- 어떤 경우에는 등급이 달라도 가격이 크게 겹치는가?

숫자보다 **패턴과 느낌**을 중심으로 정리해 보자.

> 📌 **출제 의도**
> 그래프를 보고 “어떤 등급이 가격을 가르는지, 어디서 겹치는지”를 말로 설명하는 훈련입니다. 수치보다 패턴 읽기가 우선입니다.

### 4-1. cut 등급별 분포

In [ ]:
hs_plot.boxplot(
    df2, x="cut", y="price", order=["Fair", "Good", "Very Good", "Premium", "Ideal"]
)

#### 인사이트

- cut 등급에 따라 가격 분포의 위치와 전체적인 형태가 서로 다르게 나타난다.
- Premium 컷은 다른 등급에 비해 가격 중앙값이 상대적으로 높은 위치에 형성되어 있다.
- Ideal 컷은 고급 등급임에도 불구하고 가격 중앙값이 비교적 낮은 편에 위치한다.
- Fair, Good, Very Good 등급은 중앙값이 유사한 범위에 모여 있어 평균적인 가격 수준 차이가 크지 않게 느껴진다.
- 전반적으로 cut 등급 간 중앙값 차이는 존재하지만 매우 뚜렷한 수준은 아니다.
- 모든 cut 등급에서 가격 분포의 폭이 넓게 나타나 등급 내부 가격 변동성이 크다.
- Premium과 Very Good 등급은 특히 가격 분산이 커 다양한 가격대가 함께 분포한다.
- Ideal 컷은 다른 등급에 비해 가격 분포가 상대적으로 하단 구간에 더 많이 집중된 인상을 준다.
- cut 등급 간 가격 분포가 상당 부분 겹쳐 있어 cut만으로 가격을 명확히 구분하기는 어렵다.
- 낮은 cut 등급인 Fair와 Good에서도 고가 다이아몬드가 다수 관측된다.
- 높은 cut 등급인 Ideal과 Premium에서도 저가 다이아몬드가 함께 존재한다.
- cut 등급이 높아질수록 가격이 일관되게 상승하는 단순한 서열 구조는 관찰되지 않는다.
- cut은 가격에 영향을 주는 요인 중 하나이지만 단독으로 가격을 결정하는 변수로 보이지 않는다.
- cut 등급 간 차이보다 각 등급 내부에서의 가격 변동 폭이 더 크게 느껴진다.
- 따라서 cut은 가격을 직접 구분하는 기준이라기보다 다른 품질 요인과 함께 작용하는 변수로 해석된다.

### 4-2. color 등급별 분포

In [ ]:
hs_plot.boxplot(df2, x="color", y="price", order=["J", "I", "H", "G", "F", "E", "D"])

#### 인사이트

- color 등급에 따라 가격 분포의 위치와 범위가 점진적으로 변화하는 패턴이 관찰된다.
- J에서 D로 갈수록 가격 중앙값이 전반적으로 낮아지는 경향이 나타난다.
- 낮은 색상 등급(J, I)은 중앙값이 상대적으로 높고 가격 분포의 상단이 넓게 형성되어 있다.
- 높은 색상 등급(D, E)은 중앙값이 낮은 위치에 형성되어 있으며 상대적으로 저가 구간에 밀집된 인상을 준다.
- color 등급 간 중앙값 차이는 cut에 비해 비교적 일관된 방향성을 가진다.
- 모든 color 등급에서 가격 분포의 폭이 넓어 등급 내부 가격 변동성이 크게 나타난다.
- 특히 J, I, H 등급은 가격 분산이 크고 상위 이상치가 많이 관측된다.
- D와 E 등급에서도 고가 관측치가 다수 존재하여 가격 분포가 완전히 분리되지는 않는다.
- 인접한 color 등급 간 가격 분포가 상당 부분 겹쳐 있어 색상만으로 가격을 명확히 구분하기 어렵다.
- 낮은 color 등급에서도 고가 다이아몬드가 존재하고 높은 color 등급에서도 저가 다이아몬드가 함께 존재한다.
- color 등급은 가격에 영향을 주는 요인이지만 단독으로 가격을 결정하는 변수로 보이지 않는다.
- color 등급 간 차이보다 각 등급 내부의 가격 변동 폭이 더 크게 느껴진다.
- 전반적으로 color는 가격의 수준을 완만하게 이동시키는 요인이라는 인상을 준다.

### 4-3. clarity 등급별 분포

In [ ]:
hs_plot.boxplot(
    df2,
    x="clarity",
    y="price",
    order=["I1", "SI2", "SI1", "VS2", "VS1", "VVS2", "VVS1", "IF"],
)

#### 인사이트

- clarity 등급에 따라 가격 분포의 위치와 폭이 뚜렷하게 달라지는 패턴이 관찰된다.
- SI2와 I1 등급은 가격 중앙값이 상대적으로 높은 편에 위치하며 분포 상단이 넓게 형성되어 있다.
- SI1, VS2, VS1 구간으로 이동하면서 중앙값이 점진적으로 낮아지는 경향이 나타난다.
- VVS2, VVS1, IF 등급은 중앙값이 낮은 위치에 형성되어 있으며 저가 구간에 더 밀집된 인상을 준다.
- clarity 등급이 높아질수록 가격이 일관되게 상승하는 단순한 구조는 관찰되지 않는다.
- 중간 등급(SI, VS 계열)은 가격 분포의 폭이 매우 넓어 내부 변동성이 크게 나타난다.
- 낮은 clarity 등급(I1, SI2)에서도 매우 고가의 다이아몬드가 다수 관측된다.
- 높은 clarity 등급(VVS, IF)에서도 저가 다이아몬드가 함께 존재한다.
- 인접한 clarity 등급 간 가격 분포가 상당 부분 겹쳐 있어 등급만으로 가격을 명확히 구분하기 어렵다.
- clarity 등급 간 중앙값 차이보다 각 등급 내부의 가격 변동 폭이 더 크게 느껴진다.
- 특히 SI와 VS 계열은 상위 이상치가 많이 분포하여 가격 범위가 넓게 퍼져 있다.
- 전반적으로 clarity는 가격 수준을 일정 방향으로 이동시키기보다는 분포의 폭과 겹침을 크게 만드는 요인으로 보인다.
- clarity는 가격에 영향을 주는 중요한 품질 변수이지만 단독 결정 요인으로 작용하지는 않는다는 인상을 준다.

### 4-4. 결과 종합

- color, carat, clarity 세 변수는 모두 가격에 영향을 주지만 영향 방식과 강도는 서로 다르게 나타난다.
- carat은 가격 분포의 위치와 범위를 가장 크게 이동시키는 변수로 보이며 가격 수준을 좌우하는 핵심 축으로 작용한다.
- carat이 증가할수록 가격 분포 전체가 상향 이동하고 분산도 함께 커지는 패턴이 관찰된다.
- color는 등급이 낮아질수록 중앙값이 높아지는 비교적 일관된 방향성을 보이지만 분포 간 겹침이 크다.
- color 등급은 가격의 절대적 수준보다는 가격 분포를 완만하게 이동시키는 보조적 요인으로 보인다.
- clarity는 등급에 따른 단순한 가격 서열보다는 분포의 폭과 겹침을 크게 만드는 특성이 두드러진다.
- 낮은 clarity 등급에서도 고가 다이아몬드가 다수 존재하며 높은 clarity 등급에서도 저가 다이아몬드가 함께 나타난다.
- 세 변수 모두 단독으로 가격을 명확히 구분하기에는 분포 겹침이 매우 크다.
- 특히 color와 clarity는 등급 간 중앙값 차이보다 등급 내부 가격 변동 폭이 더 크게 느껴진다.
- carat이 비슷한 경우에도 color와 clarity 조합에 따라 가격 분포가 넓게 퍼진다.
- 따라서 가격은 하나의 품질 변수에 의해 결정되기보다는 carat을 중심으로 color와 clarity가 함께 작용한 결과로 보인다.
- 세 변수의 관계는 단순 가산 구조라기보다 서로 보완적으로 가격 분포를 형성하는 다차원적 구조를 가진다.
- 종합적으로 볼 때 carat은 가격의 규모를 결정하고 color와 clarity는 그 안에서 가격의 분산과 겹침을 조정하는 역할을 한다.

## 미션 5. **“Premium 컷은 정말 더 비쌀까?”**

다음과 같은 질문을 통계적으로 검증해 보자.

> Premium 컷 다이아몬드는 Ideal 컷보다 평균 가격이 높을까?

이를 위해
- 가설을 직접 세우고
- 어떤 검정 방법이 적절한지 고민한 뒤
- 실제로 검정을 수행해 보자.

검정 결과를 해석할 때는
“유의하다 / 유의하지 않다”에서 멈추지 말고,
**이 차이가 얼마나 의미 있는 차이인지**까지 함께 생각해 보자.

> 📌 **출제 의도**
> 가설검정을 “있다/없다”로 끝내지 않고, 차이의 크기와 의미를 함께 이야기하는 연습입니다. 방법 선택 이유도 명확히 해보세요.

### 5-1. 가설 설정

| 구분 | 수식 | 내용 |
|---|---|---|
| Premium 컷 다이아몬드의 평균가격 | $\mu(Premium)$ | |
| Ideal 컷 다이아몬드의 평균가격 | $\mu(Ideal)$ | |
| 귀무가설($H_0$) | $\mu(Premium) = \mu(Ideal)$ | 두 그룹의 가격 평균은 차이가 없다. |
| 대립가설($H_1$) | $\mu(Premium) \neq \mu(Ideal)$ | 두 그룹의 가격 평균은 차이가 있다. |

In [ ]:
premium = df2.loc[origin["cut"] == "Premium", "price"]
ideal = df2.loc[origin["cut"] == "Ideal", "price"]
hs_stats.ttest_ind(x=premium, y=ideal)

In [ ]:
df_temp = df2.loc[
    (origin["cut"] == "Premium") | (origin["cut"] == "Ideal"), ["cut", "price"]
]
df_temp["cut"] = df_temp["cut"].astype("str")
df_temp

In [ ]:
hs_plot.pvalue1_anotation(df_temp, target="price", hue="cut")

#### 인사이트

- Premium 컷과 Ideal 컷의 평균 가격 차이에 대해 양측 검정(two-sided)에서 p-value가 매우 작아 두 집단의 평균이 같다는 귀무가설은 기각된다.
- 양측 검정 결과 Premium 컷과 Ideal 컷의 평균 가격은 통계적으로 유의미하게 서로 다르다.
- 단측 검정(μ(Premium) > μ(Ideal))에서 p-value가 매우 작아 Premium 컷의 평균 가격이 Ideal 컷보다 크다는 가설이 통계적으로 지지된다.
- 단측 검정(μ(Premium) < μ(Ideal))에서는 귀무가설을 기각하지 못해 Premium 컷의 평균 가격이 Ideal 컷보다 작다고 말할 근거는 없다.
- **모든 검정에서 t 통계량은 동일하게 양의 값으로 나타나 Premium 컷의 평균 가격이 Ideal 컷보다 높은 방향임을 일관되게 보여준다.**
- 검정 결과는 Premium 컷과 Ideal 컷 간 평균 가격 차이가 우연에 의해 발생했을 가능성이 극히 낮음을 시사한다.
- 통계적 유의성 관점에서 Premium 컷의 평균 가격이 Ideal 컷보다 높다는 결론은 매우 강하게 뒷받침된다.
- 이 결과는 평균 가격의 크기 비교에 대한 것이며 개별 다이아몬드 가격 분포의 겹침 여부를 직접적으로 의미하지는 않는다.
- p-value가 매우 작다는 사실은 차이의 존재를 보여주지만 그 차이의 크기나 실질적 의미 자체를 직접적으로 설명하지는 않는다.
  

## 미션 6. **“모든 컷은 서로 다를까?”**

이번에는 컷 등급 전체를 놓고 생각해 보자.

`cut`에 따른 가격 차이가
전반적으로 존재하는지 분산분석으로 확인해 보고,
차이가 있다면 **어떤 컷들 사이에서 차이가 발생하는지**
사후검정을 통해 살펴보자.

이 결과를
“가격 서열표”처럼 정리해 보는 것도 좋다.

> 📌 **출제 의도**
> 전체 차이 확인(ANOVA)에서 그치지 않고, 어떤 조합에서 차이가 나는지 구체화하는 연습입니다. 결과를 서열/지도처럼 정리해 보세요.

### 6-1. ANOVA 수행

In [ ]:
anova, a_report, post, p_report = hs_stats.oneway_anova(df2, dv="price", between="cut")

display(anova)
print(a_report)
display(post)
print(p_report)

#### 인사이트

- Welch 일원분산분석 결과, cut에 따른 평균 가격 차이는 통계적으로 유의하며,
  이는 cut 등급 간 평균 가격이 동일하다는 귀무가설이 기각됨을 의미한다.
- ANOVA의 효과크기(η²p≈0.018)는 cut 효과가 통계적으로는 유의하지만,
  전체 가격 변동을 설명하는 비중은 크지 않음을 보여준다.
- Games-Howell 사후검정 결과 10개 조합 중 9개에서 유의한 평균 차이가 확인되었으며,
  Good–Very Good 조합에서는 유의한 차이가 관찰되지 않았다.
- Premium 컷은 Ideal 및 Very Good 컷보다 평균 가격이 유의하게 높다.
- Ideal 컷은 Good 및 Very Good 컷보다 평균 가격이 유의하게 낮다.
- Fair 컷은 Ideal·Very Good·Premium 컷보다 평균 가격이 유의하게 높게 나타나,
  cut 등급 간 가격 차이가 단조적 서열 구조를 형성하지 않음을 시사한다.
- 효과크기(Hedges g)는 대부분 0.1~0.3 수준으로,
  cut에 따른 평균 차이는 실질적으로는 크지 않은 편이다.
- 이는 통계적으로 유의한 차이가 존재하더라도 실제 가격 분포의 겹침이 클 가능성을 시사하며,
  cut은 가격을 부분적으로 설명하는 요인으로서 carat 등 다른 변수와 함께 고려될 필요가 있다.


### 6-2. 컷별 가격 차이 정리

In [ ]:
# 컷별 가격 요약 및 서열 정리 (중앙값 기준)
cut_stats = (
    df2.groupby("cut")["price"]
    .agg(["count", "mean", "median"])
    .sort_values(by="mean", ascending=False)
)
cut_stats

#### 인사이트 (정정 반영)

- cut 등급별 평균 가격 기준으로 보면 Fair 컷이 가장 높은 가격 수준을 보이며, Ideal 컷이 가장 낮은 가격 수준을 가진다.
- 평균 가격 서열은 Fair → Premium → Good → Very Good → Ideal 순으로 나타난다.
- 중앙값 기준에서도 Fair 컷이 가장 높고 Ideal 컷이 가장 낮아, 평균과 중앙값 모두에서 Ideal 컷의 가격 수준이 가장 낮음을 확인할 수 있다.
- Premium과 Fair 컷은 중앙값 기준으로 매우 유사한 수준에 위치해 있으나, 평균에서는 Fair가 더 높게 나타난다.
- Premium 컷은 중앙값에 비해 평균이 상대적으로 높아, 고가 관측치가 평균을 끌어올리는 구조일 가능성을 시사한다.
- Fair 컷 역시 평균과 중앙값 간 차이가 존재하여 가격 분포의 상단 꼬리가 존재할 가능성을 보여준다.
- Very Good과 Good 컷은 평균과 중앙값 모두 중간 수준에 위치하며 서로 유사한 가격대를 형성한다.
- Ideal 컷은 표본 수가 가장 많음에도 평균과 중앙값이 모두 가장 낮아, 저가 구간에 관측치가 상대적으로 많이 분포해 있음을 보여준다.
- cut 등급별 표본 수 차이가 매우 크며, 이는 단순 평균 비교 시 분포 구조를 함께 고려해야 함을 시사한다.
- 평균 기준 가격 서열과 중앙값 기준 가격 서열은 완전히 일치하지 않으며, 이는 cut 등급별 가격 분포의 비대칭성과 이상치 영향이 서로 다름을 의미한다.
- 가격 서열표는 cut에 따른 전반적인 가격 수준을 요약해 주지만, 등급 간 가격 겹침과 분산 차이를 반영하지는 못한다.
- 따라서 cut에 따른 가격 비교는 평균 서열뿐 아니라 분포 형태와 함께 해석하는 것이 필요하다.

## 미션 7. **“품질 요인들은 서로 영향을 주고받을까?”**

컷(cut)과 색(color),
혹은 컷과 투명도(clarity)를 함께 고려하면
가격 구조가 달라질까?

이원 분산분석을 통해
- 각각의 요인이 가격에 미치는 영향
- 두 요인이 함께 작용할 때의 효과
를 구분해 보자.

결과를 해석할 때는
“상호작용이 있다/없다”를 넘어
**그 의미가 무엇인지**를 설명해 보자.

> 📌 **출제 의도**
> 요인들이 단독으로만 작용하지 않을 수 있음을 체험시키려는 단계입니다. 상호작용이 의미하는 바를 말로 풀어내는 것이 핵심입니다.

### 7-1. cut + color 분산분석

In [ ]:
anova_df, a_report, post, p_report = hs_stats.twoway_anova(
    df2, dv="price", factor_a="cut", factor_b="color"
)

display(anova_df)
print(a_report)
display(post)
print(p_report)

#### 인사이트

- cut 단독 효과는 통계적으로 유의하여 컷 등급에 따라 평균 가격 차이가 존재함을 보여준다.
- color 단독 효과 역시 통계적으로 유의하며, 효과크기(η²p)가 cut보다 상대적으로 커
  색상이 가격에 미치는 영향이 비교적 더 큼을 시사한다.
- cut과 color의 상호작용 효과도 유의하게 나타나 두 요인이 가격에 완전히 독립적으로 작용하지 않음을 보여준다.
- 다만 상호작용의 효과크기(η²p≈0.002)는 매우 작아,
  상호작용이 가격 형성의 주된 결정 요인은 아님을 시사한다.
- 이는 가격이 주로 cut과 color의 개별 효과에 의해 형성되되,
  일부 cut–color 조합에서 그 영향력이 조정됨을 의미한다.
- 동일한 cut 등급이라도 color에 따라 가격 차이가 일정하지 않을 수 있으며,
  동일한 color 등급에서도 cut에 따른 가격 상승 폭이 균등하지 않을 수 있다.
- 상호작용의 유의성은 모든 조합에서 큰 차이가 존재함을 의미하는 것은 아니며,
  특정 조합에서만 차이가 두드러짐을 의미한다.
- 전체 변동의 상당 부분은 cut과 color로 설명되지 않아,
  carat과 같은 물리적 변수나 다른 품질 요인이 가격 형성에 더 큰 역할을 할 가능성을 시사한다.
- 종합적으로 가격 구조는 단일 품질 요인의 합이 아니라
  여러 품질 요인이 함께 작용하는 다차원적 구조로 이해하는 것이 적절하다.

### 7-2. cut + clarity 이원 분석

In [ ]:
anova_df, a_report, post, p_report = hs_stats.twoway_anova(
    df2, dv="price", factor_a="cut", factor_b="clarity"
)

display(anova_df)
print(a_report)
display(post)
print(p_report)

#### 인사이트

- cut 단독 효과는 통계적으로 유의하여 컷 등급에 따라 평균 가격 차이가 존재함을 보여준다.
- clarity 단독 효과는 통계적으로 매우 유의하며,
  효과크기(η²p)가 cut보다 크게 나타나 투명도가 가격에 미치는 영향이 상대적으로 더 큼을 시사한다.
- cut과 clarity의 상호작용 효과도 유의하게 나타나
  두 요인이 가격에 완전히 독립적으로 작용하지 않음을 보여준다.
- 다만 상호작용의 효과크기(η²p≈0.006)는
  cut과 clarity의 단독 효과에 비해 작아 상호작용이 가격을 지배하는 주된 요인은 아님을 시사한다.
- 이는 가격이 주로 clarity와 cut의 개별 효과에 의해 형성되되,
  일부 cut–clarity 조합에서 그 영향력이 조정됨을 의미한다.
- 동일한 cut 등급이라도 clarity 수준에 따라 가격 차이가 일정하지 않을 수 있으며,
  동일한 clarity 등급에서도 cut에 따라 가격 상승 폭이 균등하지 않을 가능성이 있다.
- 상호작용의 유의성은 모든 cut–clarity 조합에서 큰 차이가 발생함을 의미하는 것은 아니며,
  특정 조합에서만 차이가 두드러짐을 의미한다.
- 전체 가격 변동의 상당 부분은 cut과 clarity로 설명되지 않아,
  carat과 같은 물리적 변수나 다른 품질 요인이 가격 형성에 더 큰 역할을 할 가능성을 시사한다.
- 종합적으로 clarity는 cut보다 가격 설명력이 더 큰 품질 요인으로 보이며,
  cut은 clarity의 효과를 일부 조정하는 역할을 한다.
- 따라서 가격 구조는 cut과 clarity가 단순히 더해지는 것이 아니라,
  조합에 따라 미세하게 달라지는 다차원적 구조로 이해하는 것이 적절하다.

## 미션 8. **“가격과 가장 가까운 물리적 변수는?”**

연속형 변수들 사이의 상관관계를 계산해 보자.

`price`와 `carat`, `x`, `y`, `z`, `depth`, `table` 중
어떤 변수가 가장 강한 관계를 보이는지 확인해 보고,
왜 그런 결과가 나왔는지 팀의 언어로 설명해 보자.

이 과정에서
Pearson과 Spearman 중 어떤 상관계수가 더 적절한지도 함께 고민해 보자.

> 📌 **출제 의도**
> 연속형 변수 관계를 요약할 때 어떤 계수를 쓰고, 왜 그 선택이 타당한지 설명하는 연습입니다. 물리적 의미와 수치가 만나는 지점을 찾아보세요.

### 상관 분석 수행

In [ ]:
result, corr_metric = hs_stats.corr_pairwise(df2)

display(result)
hs_plot.heatmap(corr_metric)

#### 인사이트

- price는 carat, x, y, z와 매우 강한 양의 상관관계를 보이며(Spearman ρ≈0.96), 가격이 크기·치수 변수들과 단조적으로 강하게 연관되어 있음을 확인할 수 있다.
- carat과 x, y, z 사이의 상관계수는 0.99 내외로 매우 높아, carat과 물리적 치수 변수들이 거의 동일한 정보 구조를 공유하고 있음을 보여준다.
- x, y, z 상호 간에도 상관계수가 0.98~0.99 수준으로 나타나, 세 치수 변수는 강하게 결합된 하나의 묶음으로 관찰된다.
- price와 depth의 상관계수는 0.01 수준으로 매우 약하며, 통계적으로는 유의하지만 실질적인 연관성은 거의 없다.
- price와 table의 상관계수는 약 0.17로, 통계적으로 유의하나 강도는 약한 수준에 머문다.
- carat과 depth의 상관계수는 약 0.03으로 매우 약한 양의 상관을 보이며, carat과 table 역시 약한 양의 상관(ρ≈0.19)에 그친다.
- x, y는 depth와 약한 음의 상관을 보이고(z는 약한 양의 상관), 치수 변수와 depth 간의 관계는 전반적으로 약하다.
- depth와 table 사이에는 약한 음의 상관관계(ρ≈-0.25)가 관찰되어, 두 비율 변수 간에도 단조 관계가 크지 않음을 보여준다.
- 대부분의 약한 상관관계는 표본 수가 매우 크기 때문에 통계적으로 유의하나, 상관 강도 기준에서는 실질적 영향이 제한적이다.
- 전체적으로 price와 강하게 연관된 변수들은 carat 및 x·y·z로 한정되며, depth와 table은 가격 및 크기 변수들과의 연관성이 상대적으로 약하다.

## 미션 9. **“가격을 설명하는 회귀모형을 만들어보자”**

이제 지금까지 살펴본 변수들을 활용해
`log(price)`를 종속변수로 하는 다중선형회귀모형을 만들어보자.

변수를 선택할 때는
“넣을 수 있어서”가 아니라
**“설명하고 싶어서”** 선택했는지 스스로 점검해 보자.

회귀계수 하나하나가
가격 구조를 어떻게 설명해 주는지 말로 풀어 써보자.

> 📌 **출제 의도**
> 변수 선택과 모델 설계를 “설명하고 싶은 세계”에 맞춰보는 연습입니다. 계수를 단순 숫자가 아닌 가격 구조의 언어로 바꾸는 것이 목표입니다.

### 9-1. Base Model 구축

로그 처리 전 데이터프레임에 더미 변수만 생성하고 모든 변수를 그대로 넣고 분석

In [ ]:
df_dummy1 = hs_prep.get_dummies(df1, "cut", "color", "clarity")
fit1, result1 = hs_stats.ols(df_dummy1, yname="price", report="summary")
display(result1)

### 9-2. 로그 적용

In [ ]:
df_dummy2 = hs_prep.get_dummies(df2, "cut", "color", "clarity")
fit2, result2 = hs_stats.ols(df_dummy2, yname="price", report="summary")
result2

> 로그 적용이 설명력이 더 증가한다.

### 9-3. 교효작용 추가

In [ ]:
df3 = hs_prep.add_interaction(df2, [("cut", "clarity"), ("cut", "color")])
df3

In [ ]:
df_dummy3 = hs_prep.get_dummies(
    df3, "cut", "color", "clarity", "cut_clarity", "cut_color"
)
fit3, result3 = hs_stats.ols(df_dummy3, yname="price", report="summary")
result3

> 교효작용에 대한 모형 개선은 없다.

### 9-3. 다중공선성 제거

In [ ]:
df4 = hs_stats.vif_filter(df3, yname="price", verbose=True)
df4

In [ ]:
df_dummy4 = hs_prep.get_dummies(
    df4, "cut", "color", "clarity", "cut_clarity", "cut_color"
)

fit4, result4 = hs_stats.ols(df_dummy4, yname="price", report="summary")

display(result4)

### 9-4. 모형 비교

In [ ]:
result_df = concat([result1, result2, result3, result4])
result_df.insert(
    0, "method", ["기본모형", "로그변환", "교효작용 추가", "다중공선성 제거"]
)
result_df.reset_index(inplace=True, drop=True)
result_df.index = result_df.index + 1
result_df

### 9-4. 최종 모형으로 3번 선택

최종 모형으로는 **모형 3(교효작용 추가)**를 선택하는 것이 가장 합리적이다.

- 모형 3은 R²=0.983으로 모형 2와 동일한 설명력을 유지하면서, 단순 변환 효과를 넘어 변수 간 관계 구조를 반영한다.
- 모형 2(로그변환)는 설명력은 가장 높지만, 변수 간 상호작용이나 조건부 효과를 반영하지 못한 순수 변환 모형이다.
- 모형 3은 교호작용을 포함함으로써 주효과만으로 설명되지 않는 이질적 효과를 구조적으로 포착한다.
- 모형 1(기본모형)은 설명력은 높지만(0.970), 로그변환·교호작용을 반영한 확장 모형 대비 정보 활용도가 낮다.
- 모형 4(다중공선성 제거)는 공선성 안정성은 개선되었으나(R²=0.914), 설명력 손실이 매우 커 최종 모형으로는 부적절하다.
- Durbin–Watson 통계량은 모형 3이 1.279로 완벽하지는 않으나, 모형 2와 유사하며 해석을 저해할 수준은 아니다.
- 종합적으로 모형 3은 **높은 설명력(R²), 구조적 해석 가능성, 과도한 정보 손실 없음**을 동시에 충족한다.

## 미션 10. **“이 회귀모형은 믿을 만할까?”**

회귀분석 결과를 그대로 받아들이기 전에,
가정이 얼마나 잘 지켜졌는지 확인해 보자.

잔차 그림과 진단 지표를 통해
- 어떤 가정이 잘 지켜졌는지
- 어떤 부분이 아쉬운지
를 정리해 보자.

그리고 이 모형을 더 개선하려면
무엇을 해볼 수 있을지도 함께 제안해 보자.

> 📌 **출제 의도**
> 성능만 보는 것이 아니라, 회귀 가정과 진단을 통해 “이 모델을 어디까지 믿을 수 있는가”를 점검하고 개선 아이디어를 제시하게 합니다.

### 10-1. 선형성 검정

In [ ]:
result = hs_stats.ols_linearity_test(fit3)
display(result)

hs_plot.ols_residplot(fit3, lowess=True, mse=True)

#### 인사이트

- Ramsey RESET 검정 결과, 현재 회귀모형에서 함수 형태의 심각한 지정 오류나 비선형성 누락을 의심할 통계적 근거는 발견되지 않았다.

### 10-2. 정규성 검정

In [ ]:
result = hs_stats.ols_normality_test(fit3)
display(result)

hs_plot.ols_qqplot(fit3)

#### 인사이트

- Jarque–Bera 검정에서 p-value = 0.0000으로 나타나, 유의수준 0.05에서 잔차가 정규분포를 따른다는 귀무가설은 기각된다.
- 이는 회귀모형의 잔차 분포가 왜도(skewness) 또는 첨도(kurtosis) 측면에서 정규분포와 유의하게 다름을 의미한다.
- 검정통계량이 17307로 매우 큰 값을 보인 점은, 표본 수가 크고 잔차 분포의 비정규성이 통계적으로 강하게 검출되었음을 시사한다.
- 정규성 위반은 **모형의 계수 추정 자체(OLS의 불편성)**를 무효화하지는 않지만, 통계적 추론(p-value, 신뢰구간)의 엄밀성에는 영향을 줄 수 있다.
- 특히 대규모 표본에서는 미세한 비정규성도 쉽게 유의하게 검출되므로, 결과 해석 시 검정 결과를 과도하게 해석할 필요는 없다.

#### 시사점

- 본 회귀모형은 정규성 가정을 엄밀히 충족하지는 않으므로, 계수 유의성 해석 시 주의가 필요하다.
- 계수 해석이 핵심 목적일 경우, **강건 표준오차(robust standard errors)**를 사용해 추론의 신뢰성을 보완하는 것이 합리적이다.
- 정규성 위반이 심각하다고 판단될 경우, **종속변수 변환(예: 로그 변환)**이나 이상치 영향 점검을 추가적으로 고려할 수 있다.
- 표본 수가 매우 큰 분석 맥락에서는, 정규성 가정보다 선형성·독립성·공선성 관리가 실질적으로 더 중요한 진단 기준이 될 수 있다.
- 따라서 본 결과는 모형을 폐기해야 한다는 신호라기보다는, 추론 방법을 보완하거나 해석의 엄밀성을 높여야 한다는 경고 신호로 해석하는 것이 적절하다.

### 10-3. 등분산성 검정

In [ ]:
hs_stats.ols_variance_test(fit3)

#### 인사이트

- Breusch–Pagan 검정과 White 검정 모두에서 p-value가 0.05보다 작아, 잔차의 분산이 일정하다는 귀무가설은 기각된다.
- 두 검정이 동시에 유의하게 나타났다는 점은, 단순한 선형 이분산뿐 아니라 보다 일반적인 형태의 이분산성까지 관측되었음을 의미한다.
- 검정통계량(LM)이 매우 큰 값으로 보고된 것은 표본 수가 크고, 이분산 패턴이 통계적으로 명확히 감지되었음을 보여준다.
- 등분산성 위반은 OLS 계수의 불편성 자체를 무효화하지는 않지만, 표준오차·t통계량·p-value의 신뢰성에는 영향을 미친다.
- 따라서 현재 모형에서 계수의 유의성 판단은 기본 OLS 표준오차를 그대로 사용할 경우 과대·과소 추정될 가능성이 있다.

#### 시사점

- 계수 해석이 목적이라면 강건 표준오차(HC 계열)를 적용하여 추론의 신뢰성을 보완하는 것이 필요하다.
- 이분산 구조가 체계적이라고 판단될 경우, 종속변수 변환(예: 로그 변환)이나 가중회귀(WLS/GLS)를 대안으로 고려할 수 있다.
- 본 결과는 모형이 잘못되었다기보다는, 오차 구조에 대한 추가적인 보정 없이 통계적 추론을 수행하기에는 부적절하다는 신호로 해석하는 것이 타당하다.

### 10-4. 독립성 검정

In [ ]:
hs_stats.ols_independence_test(fit3)

#### 인사이트

- Durbin–Watson 통계량이 1.279로 2보다 상당히 작은 값으로 나타나, 잔차 간 **양의 자기상관 가능성**이 관측된다.
- 일반적으로 DW 값이 2에 가까울수록 독립성이 만족되며, 1.5 미만일 경우 **독립성 가정 위반을 의심**할 수 있다.
- 본 결과는 회귀모형의 잔차가 서로 독립적이라는 귀무가설을 지지하지 못하며, **연속된 관측치 간 오차가 체계적으로 연관**되어 있음을 시사한다.
- 독립성 위반은 OLS 계수의 불편성 자체를 무효화하지는 않지만, **표준오차와 검정통계량의 신뢰성을 저하시킬 수 있다**.
- 특히 표본이 순서성을 가지는 구조(시간 순서, 정렬된 관측치 등)라면, 자기상관이 통계적으로 쉽게 검출될 수 있다.
- 시사점으로, 계수 해석이 목적일 경우 **강건 표준오차(Newey–West 등)**를 사용하거나, 오차의 상관 구조를 명시적으로 반영한 **GLS 계열 모형**을 고려할 필요가 있다.
- 본 결과는 모형 자체가 부적절하다는 의미라기보다, **오차 독립성 가정을 그대로 두고 추론을 수행하기에는 주의가 필요함을 알리는 신호**로 해석하는 것이 적절하다.